In [1]:
print('Ok')

Ok


In [25]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain_pinecone import PineconeVectorStore
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers

In [5]:
from tqdm.autonotebook import tqdm

In [18]:
import os
os.environ["PINECONE_API_KEY"] = "pcsk_FpWhw_Mdvah7fnDnSTWhiBP3jdEGoBKngWXD9XwYR4ZToLpXBZA5JJGmmdGu3uQecNESH"
os.environ["PINECONE_API_ENV"] = "us-east-1"

In [9]:
#Extract Data from the PDF
def load_pdf(data):
    loader=DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

    

In [10]:
extracted_data = load_pdf("data/")

In [12]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [13]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 5860


In [30]:
text_chunks[0]

Document(metadata={'source': 'data\\Medical_book.pdf', 'page': 1, 'page_label': '2'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION')

In [14]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [15]:
embeddings = download_hugging_face_embeddings()

C:\Users\Suraj\AppData\Local\Temp\ipykernel_11628\1337643473.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\Suraj\.conda\envs\myenv\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Suraj\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by set

In [16]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [17]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [20]:
pc = pinecone.Pinecone(api_key=os.environ["PINECONE_API_KEY"])

In [21]:
print(pc.list_indexes())

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 1024,
              'host': 'testing-smz119p.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'testing',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}},
             {'deletion_protection': 'disabled',
              'dimension': 384,
              'host': 'medibot-smz119p.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'medibot',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}


In [22]:
index_name='medibot'
index=pc.Index(index_name)

In [32]:
vectorstore_from_docs = PineconeVectorStore.from_documents(
        text_chunks,
        index_name=index_name,
        embedding=embeddings
    )

In [36]:
query = "What are allergies?"
vectorstore_from_docs.similarity_search(query, k=3)

[Document(id='eb0b0f91-4857-40e6-a2ec-e42ba2388e6b', metadata={'page': 129.0, 'page_label': '130', 'source': 'data\\Medical_book.pdf'}, page_content='reaction. Allergic rhinitis is characterized by an itchy,\nrunny nose, often with a scratchy or irritated throat due\nto post-nasal drip. Inflammation of the thin membrane\ncovering the eye (allergic conjunctivitis) causes redness,\nirritation, and increased tearing in the eyes. Asthma caus-\nes wheezing, coughing, and shortness of breath. Symp-\ntoms of food allergies depend on the tissues most sensi-\ntive to the allergen and whether the allergen spread sys-'),
 Document(id='80b584f4-1067-4c77-b5f4-b07bceb1e888', metadata={'page': 135.0, 'page_label': '136', 'source': 'data\\Medical_book.pdf'}, page_content='reactions is triggered by harmless, everyday substances.\nThis is the condition known as allergy, and the offend-\ning substance is called an allergen. Common inhaled\nallergens include pollen, dust, and insect parts from tiny\nhous

In [37]:

prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""


In [38]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=['context','question'])
chain_type_kwargs={"prompt":PROMPT}

In [39]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type='llama',
                  confif={'max_new_tokens':512,
                          'temperature':0.8})

In [41]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=vectorstore_from_docs.as_retriever(search_kwargs={'k':2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [3]:
while True:

    user_input=input(f"Input Prompt(Type 'Exit' to cancel): ")
    if user_input=='Exit':
        break
    else:
        result=qa({'query':user_input})
        print('Response: ',result['result'])